In [1]:
!pip install -q rouge_score
!pip install -q evaluate

In [3]:
import pandas as pd
import numpy as np
import random
import math
from PIL import Image
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
import tensorflow as tf
from transformers import pipeline
import cv2
import json
import os
import zipfile
from transformers import pipeline
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import default_data_collator
from transformers import default_data_collator
import evaluate
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from PIL import Image
import datasets
from transformers import VisionEncoderDecoderModel, AutoFeatureExtractor,AutoTokenizer
import torchvision.transforms as transforms
from nltk.translate.bleu_score import corpus_bleu
os.environ["WANDB_DISABLED"] = "true"
ignore_pad_token_for_loss = True
metric = evaluate.load("rouge")
import nltk
try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    nltk.download("punkt", quiet=True)

2024-05-13 05:39:53.676753: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 05:39:53.676862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 05:39:53.804313: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
images_folder = r'/kaggle/input/fyp-dataset-ego4d/Images/Images'
image_path = r'/kaggle/input/fyp-dataset-ego4d/paths.txt'

In [6]:
narrations = dict()
f = open(r'/kaggle/input/fyp-dataset-ego4d/narrations.json')
narrations = json.load(f)

In [7]:
totalvids = ['062f1e55-67d8-4cdc-89c6-7fb361a9b0f9.mp4', '0d95205f-a5f2-40db-9e30-29c668fbd323.mp4',
 '0dbabadf-c3d5-4e98-bb20-ce9d930c2474.mp4', '0e529f0a-6432-4f2c-bbd1-e495c019f600.mp4',
 '1bfe5ac2-cbf8-4364-8a30-60d97dd395df.mp4', '20ef0632-9c41-43d0-aea4-60307be7431a.mp4',
 '291e4d57-380b-4f8f-81e0-a67a3c3528a4.mp4', '2e160c88-209f-4bb2-90d4-e7edf5664bce.mp4',
 '3c6ac251-4298-4421-be93-9262bcdbc320.mp4', '4466d75d-2da3-404f-b49c-f28b3886194e.mp4',
 '44a173ba-e836-4efc-b485-e94d6bbafa8e.mp4', '46af7819-8dfd-4c3c-9b21-5ad0d9095580.mp4',
 '490db405-c25f-44cb-b7c5-614ba424f7c4.mp4', '4ab8fc08-2ae6-4b27-8c72-e9700e0161bb.mp4',
 '4b97cf1d-4989-41d8-8b9e-f6b3d648d9d0.mp4', '56e92c65-c82e-413b-aaf9-26a0dd5fbbe2.mp4',
 '575dbff3-44c8-4fda-92f7-fbf3070f8789.mp4', '5ce0e545-7b31-4905-944c-72823f9b8bac.mp4',
 '5d2043c0-6982-4259-88bd-b4d0fc5d789a.mp4', '61767972-9303-4f55-980c-832d7a454d6e.mp4',
 '63f0f27d-645c-4efc-9162-6f0cb0353bc8.mp4', '6c0a8bc3-1a83-4f90-bb7f-6baf959b2364.mp4',
 '73005180-4af4-48a3-8c9a-f4afe2b99c58.mp4', '7bc80e9a-d235-4245-9876-9f414969b6d0.mp4',
 '82579639-34d8-4b3f-a67f-c81853c4f3c8.mp4', '82b9dd3c-5072-48cd-a8f4-9515ea5b2c13.mp4',
 '8301a3fc-aac5-466d-bb02-d0ac7a81ccc6.mp4', '8795f7e7-c1ca-4ca0-b777-982a16fa68e1.mp4',
 '8d900b59-e95d-4c9b-9432-8f5096aac2db.mp4', '9b9ed51e-37a0-43bd-82fc-32bbe702751a.mp4',
 'a3f45e46-6c63-402c-a6b9-1c3ad529166d.mp4', 'a774ee38-7f06-4733-8b76-1fb07f95dcf4.mp4',
 'c569794f-e88a-414d-b34c-a06d7f7f800a.mp4', 'cb9feccb-c0f7-42e7-a8f0-99d4f0450944.mp4',
 'cd8590b6-4c27-4932-9ccf-501380d35b14.mp4', 'ceea3a90-8339-4512-8ae2-0da5f1ee6bff.mp4',
 'e2469ed9-64a2-42b4-8cf7-089b1c7e7ba5.mp4', 'ecdc21d9-ee59-48ab-ae02-53e9aba05ef8.mp4',
 'f2b29e04-a09d-4faa-a966-6f639dca11ec.mp4', 'f5155c9c-26a8-43f5-b063-2343263dcf31.mp4',
 'f54fd12b-d439-4f3c-9d0c-293336dd33b1.mp4', 'f6fea609-23c4-431c-8ac2-484f334a3a83.mp4',
 'f7eba194-18eb-49c4-bccf-df29755ecccd.mp4', 'fd878068-4a9a-458d-9225-fe4865907f0a.mp4']

In [8]:
videoId = []
for i in range(len(totalvids)):
    a = totalvids[i][:-4]
    videoId.append(a)

In [9]:
narrations_count = {}

for Id, narr_list in narrations.items():
    if Id in videoId:
        narrations_count[Id] = len(narr_list)

for i, count in narrations_count.items():
    print(f"VideoId: {i}, Number of Narrations: {count}")

VideoId: 8301a3fc-aac5-466d-bb02-d0ac7a81ccc6, Number of Narrations: 70
VideoId: f6fea609-23c4-431c-8ac2-484f334a3a83, Number of Narrations: 199
VideoId: 062f1e55-67d8-4cdc-89c6-7fb361a9b0f9, Number of Narrations: 165
VideoId: fd878068-4a9a-458d-9225-fe4865907f0a, Number of Narrations: 118
VideoId: 3c6ac251-4298-4421-be93-9262bcdbc320, Number of Narrations: 102
VideoId: 291e4d57-380b-4f8f-81e0-a67a3c3528a4, Number of Narrations: 148
VideoId: f54fd12b-d439-4f3c-9d0c-293336dd33b1, Number of Narrations: 60
VideoId: 1bfe5ac2-cbf8-4364-8a30-60d97dd395df, Number of Narrations: 180
VideoId: 2e160c88-209f-4bb2-90d4-e7edf5664bce, Number of Narrations: 238
VideoId: 73005180-4af4-48a3-8c9a-f4afe2b99c58, Number of Narrations: 119
VideoId: 0e529f0a-6432-4f2c-bbd1-e495c019f600, Number of Narrations: 84
VideoId: a774ee38-7f06-4733-8b76-1fb07f95dcf4, Number of Narrations: 14
VideoId: 82579639-34d8-4b3f-a67f-c81853c4f3c8, Number of Narrations: 118
VideoId: ceea3a90-8339-4512-8ae2-0da5f1ee6bff, Number o

In [9]:
# code for creating image paths
path = r'/kaggle/working/'

with open('paths.txt', 'w') as f:
    for filename in os.listdir(images_folder):
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.JPEG'):
            image = os.path.join(images_folder, filename)
            f.write(image)
            f.write('\n')

In [10]:
def read_data(image_file):
    with open(image_file, 'r') as f:
        image_paths = f.readlines()
    image_paths = [path.strip() for path in image_paths]
    return image_paths


def load_images(image_paths):
    images = []
    for path in image_paths:
        image = Image.open(path)
        images.append(image)
    return images

In [11]:
image_paths = read_data(image_path)
#images = load_images(image_paths)
len(image_paths)

6500

In [12]:
image_data = []
for path in image_paths:
    path = path.strip()
    parts = path.split('_')
    video_id = parts[0].split('/')[-1]
    frame_number = int(parts[1].split('.')[0])
    image_data.append({'path': path, 'video_id': video_id, 'frame_number': frame_number})
    
narrations_dict = {}
for video_id, data in narrations.items():
    narrations_dict[video_id] = [{'frame': entry['frame'], 'text': entry['text']} for entry in data]

image_texts = []
for data in image_data:
    video_id = data['video_id']
    frame_number = data['frame_number']
    if video_id in narrations_dict:
        for entry in narrations_dict[video_id]:
            if entry['frame'] == frame_number:
                image_texts.append({'ImagePath': data['path'], 'Text': entry['text']})
                break

In [13]:
data = pd.DataFrame(image_texts)
data

,ImagePath,Text
0,/kaggle/input/ego4d-data/Images/Images/f6fea60...,#C C wipes the window
1,/kaggle/input/ego4d-data/Images/Images/8257963...,#C C walks to the table
2,/kaggle/input/ego4d-data/Images/Images/63f0f27...,#C C sips the juice
3,/kaggle/input/ego4d-data/Images/Images/cd8590b...,#C C closes the tap with his right hand.
4,/kaggle/input/ego4d-data/Images/Images/575dbff...,#C C picks panties
...,...,...
6495,/kaggle/input/ego4d-data/Images/Images/82b9dd3...,#C C puts knife in the basket
6496,/kaggle/input/ego4d-data/Images/Images/4ab8fc0...,#C C puts cloth down
6497,/kaggle/input/ego4d-data/Images/Images/ecdc21d...,#C C washes the wooden spatula
6498,/kaggle/input/ego4d-data/Images/Images/e2469ed...,#C C moves a cloth from the basket into the wa...


In [14]:
data.shape

(6500, 2)

In [15]:
train, test = train_test_split(data, test_size = 0.2)
trainDataset = Dataset.from_pandas(train)
testDataset = Dataset.from_pandas(test)
testDataset, valDataset= testDataset.train_test_split(test_size=0.2).values()

In [16]:
ds = datasets.DatasetDict({"train": trainDataset,"test":testDataset, "val":valDataset})
ds

DatasetDict({
    train: Dataset({
        features: ['ImagePath', 'Text', '__index_level_0__'],
        num_rows: 5200
    })
    test: Dataset({
        features: ['ImagePath', 'Text', '__index_level_0__'],
        num_rows: 1040
    })
    val: Dataset({
        features: ['ImagePath', 'Text', '__index_level_0__'],
        num_rows: 260
    })
})

## VISION TRANSFORMER
## Initializing Vision Encoder Decoder Model

In [17]:
image_encoder_model = "google/vit-base-patch16-224-in21k"
text_decode_model = "gpt2"

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(image_encoder_model, text_decode_model)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.crossattention.c_attn.bias', 'h.0.crossattention.c_attn.weight', 'h.0.crossattention.c_proj.bias', 'h.0.crossattention.c_proj.weight', 'h.0.crossattention.q_attn.bias', 'h.0.crossattention.q_attn.weight', 'h.0.ln_cross_attn.bias', 'h.0.ln_cross_attn.weight', 'h.1.crossattention.c_attn.bias', 'h.1.crossattention.c_attn.weight', 'h.1.crossattention.c_proj.bias', 'h.1.crossattention.c_proj.weight', 'h.1.crossattention.q_attn.bias', 'h.1.crossattention.q_attn.weight', 'h.1.ln_cross_attn.bias', 'h.1.ln_cross_attn.weight', 'h.10.crossattention.c_attn.bias', 'h.10.crossattention.c_attn.weight', 'h.10.crossattention.c_proj.bias', 'h.10.crossattention.c_proj.weight', 'h.10.crossattention.q_attn.bias', 'h.10.crossattention.q_attn.weight', 'h.10.ln_cross_attn.bias', 'h.10.ln_cross_attn.weight', 'h.11.crossattention.c_attn.bias', 'h.11.crossattention.c_attn.weight', 'h.11.crossat

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [18]:
# image feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained(image_encoder_model)
# text tokenizer
tokenizer = AutoTokenizer.from_pretrained(text_decode_model)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [19]:
# GPT2 only has bos/eos tokens but not decoder_start/pad tokens
tokenizer.pad_token = tokenizer.eos_token

# update the model config
model.config.eos_token_id = tokenizer.eos_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [20]:
output_dir = "vit-gpt-model"
model.save_pretrained(output_dir)
feature_extractor.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Removed shared tensor {'decoder.lm_head.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading


('vit-gpt-model/tokenizer_config.json',
 'vit-gpt-model/special_tokens_map.json',
 'vit-gpt-model/vocab.json',
 'vit-gpt-model/merges.txt',
 'vit-gpt-model/added_tokens.json',
 'vit-gpt-model/tokenizer.json')

## Data Preparation

In [21]:
# text preprocessing step
def tokenization_fn(captions, max_target_length):
    """Run tokenization on captions."""
    labels = tokenizer(captions, 
                      padding="max_length", 
                      max_length=max_target_length).input_ids

    return labels

# image preprocessing step
def feature_extraction_fn(image_paths, check_image=True):
    """
    Run feature extraction on images
    If `check_image` is `True`, the examples that fails during `Image.open()` will be caught and discarded.
    Otherwise, an exception will be thrown.
    """

    model_inputs = {}

    if check_image:
        images = []
        to_keep = []
        for image_file in image_paths:
            try:
                img = Image.open(image_file)
                images.append(img)
                to_keep.append(True)
            except Exception:
                to_keep.append(False)
    else:
        images = [Image.open(image_file) for image_file in image_paths]

    encoder_inputs = feature_extractor(images=images, return_tensors="np")

    return encoder_inputs.pixel_values

def preprocess_fn(examples, max_target_length, check_image = True):
    """Run tokenization + image feature extraction"""
    image_paths = examples['ImagePath']
    captions = examples['Text']    
    
    model_inputs = {}
    # This contains image path column
    model_inputs['labels'] = tokenization_fn(captions, max_target_length)
    model_inputs['pixel_values'] = feature_extraction_fn(image_paths, check_image=check_image)

    return model_inputs

In [22]:
processed_dataset = ds.map(
    function=preprocess_fn,
    batched=True,
    fn_kwargs={"max_target_length": 128},
    remove_columns=ds['train'].column_names
)

Map:   0%|          | 0/5200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

Map:   0%|          | 0/260 [00:00<?, ? examples/s]

In [34]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    output_dir="/kaggle/working/image-captioning-output",
    num_train_epochs=2,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [31]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds,
                                                     decoded_labels)

    result = metric.compute(predictions=decoded_preds,
                            references=decoded_labels,
                            use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [32]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'pixel_values'],
        num_rows: 5200
    })
    test: Dataset({
        features: ['labels', 'pixel_values'],
        num_rows: 1040
    })
    val: Dataset({
        features: ['labels', 'pixel_values'],
        num_rows: 260
    })
})

In [35]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=processed_dataset['train'],
    eval_dataset=processed_dataset['val'],
    data_collator=default_data_collator,
)

In [185]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.108700,0.090591,48.994500,27.722300,48.317200,48.403600,9.919231
2,0.073400,0.078407,54.334000,34.359200,53.537800,53.532900,10.430769


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_

TrainOutput(global_step=2600, training_loss=0.10594156320278461, metrics={'train_runtime': 2158.173, 'train_samples_per_second': 4.819, 'train_steps_per_second': 1.205, 'total_flos': 1.8768244676493312e+18, 'train_loss': 0.10594156320278461, 'epoch': 2.0})

In [186]:
trainer.save_model("/kaggle/working/image-captioning-output")

In [187]:
tokenizer.save_pretrained("/kaggle/working/image-captioning-output")

('/kaggle/working/image-captioning-output/tokenizer_config.json',
 '/kaggle/working/image-captioning-output/special_tokens_map.json',
 '/kaggle/working/image-captioning-output/vocab.json',
 '/kaggle/working/image-captioning-output/merges.txt',
 '/kaggle/working/image-captioning-output/added_tokens.json',
 '/kaggle/working/image-captioning-output/tokenizer.json')

In [62]:
# loading fine-tuned model
image_captioner = pipeline("image-to-text", model="/kaggle/input/fyp-dataset-ego4d/image-captioning-output")

## Calculating Bleu Score on total test images

In [69]:
total_test = ds['test']['ImagePath']
len(total_test)

1040

In [70]:
total_text = ds['test']['Text']
len(total_text)

1040

In [71]:
total_pred = []
for i in range(len(total_test)):
    caption = image_captioner(total_test[i])
    total_pred.append(caption)

In [75]:
total_values = [item[0]['generated_text'] for item in total_pred]

In [77]:
# Tokenize reference and candidate sentences
reference_tokens = [nltk.word_tokenize(reference_sentence.lower()) for reference_sentence in total_values]
candidate_tokens = [nltk.word_tokenize(candidate_sentence.lower()) for candidate_sentence in total_text]

# Flatten the list of tokens
reference_tokens_flat = [token for sublist in reference_tokens for token in sublist]
candidate_tokens_flat = [token for sublist in candidate_tokens for token in sublist]

# Calculate BLEU score
bleu_score = corpus_bleu([[reference_tokens_flat]], [candidate_tokens_flat])
bleu_score

0.5117226305952609

## Code to save necessary files/outputs

In [79]:
path = r'/kaggle/working/'

with open('total_predictions.txt', 'w') as f:
    for i in range(len(total_values)):
        f.write(total_values[i])
        f.write('\n')

In [80]:
path = r'/kaggle/working/'

with open('total_test_images.txt', 'w') as f:
    for i in range(len(total_test)):
        f.write(total_test[i])
        f.write('\n')

In [81]:
path = r'/kaggle/working/'

with open('original_test_text.txt', 'w') as f:
    for i in range(len(total_text)):
        f.write(total_text[i])
        f.write('\n')